### Python 3.6
### 10.10.2017

In [47]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import psycopg2 # Will need to install if you do not have it already

In [48]:
class prelimstats(object):
    """Takes in a datatable name and output different statistics about the variables. """
    
    def __init__(self, datatable):
        self.datatable = datatable
    
    def dbconnect(self):
        !mkdir $self.datatable
        %cd $self.datatable
        
        ## To connect to the database ##
        try:        
            # Specify your own personal username & password
            conn = psycopg2.connect("dbname='SQLBook' user='postgres' host='/tmp/' password='Awesome534'")
        except:
            print ('unable to connect to the database')

        self.df = pd.read_sql("select * from %s" %self.datatable, con=conn)
        conn.close()
        ##
        
        self.df = self.df.replace('NULL', np.nan) #make string 'NULL' a np.nan
        self.text_file = open("Data.txt", "w")
        self.text_file.write("Initial data has %i rows and %i columns" %(self.df.shape[0], self.df.shape[1]))
    
    def metadata_overview(self):
        try:
            self.df['date'] = pd.to_datetime(self.df['date']) # for calendar
            self.df['iso'] = pd.to_datetime(self.df['iso']) # for calendar
        except:
            pass
        
        self.cols = self.df.columns
        for col in self.cols:
            if self.df[col].nunique() == 1 and type(self.df[col].unique()[0]) == 'unicode':
                self.df[col] = self.df[col].astype(str)
                self.text_file.write("\nThe datatype for column: %s is %s." %(col, type(self.df[col].unique()[0])))
            elif self.df[col].nunique() > 1 and type(self.df[col].unique()[1]) == 'unicode':
                self.df[col] = self.df[col].astype(str)
                self.text_file.write("\nThe datatype for column: %s is %s." %(col, type(self.df[col].unique()[1])))
            else:
                try:
                    self.text_file.write("\nThe datatype for column: %s is %s." %(col, type(self.df[col].unique()[1])))
                except:
                    self.text_file.write("\nThe datatype for column: %s is %s." %(col, type(self.df[col].unique()[0])))

        self.nominal = []
        self.numeric = []
        self.datetime = []
        for col in self.cols:
            if (self.df[col].nunique() > 1 and (type(self.df[col].unique()[1]) == np.int64 or type(self.df[col].unique()[1]) == np.float64)) or (self.df[col].nunique() == 1 and (type(self.df[col].unique()[0]) == np.int64 or type(self.df[col].unique()[0]) == np.float64)):
                self.numeric.append(col)
            elif (self.df[col].nunique() > 1 and type(self.df[col].unique()[1]) == np.datetime64) or (self.df[col].nunique() == 1 and type(self.df[col].unique()[0]) == np.datetime64):
                self.datetime.append(col)
            else:
                self.nominal.append(col)
        self.text_file.write("\nNominal attribute count: %i" %(len(self.nominal)))
        self.text_file.write("\nNumeric attribute count: %i" %(len(self.numeric)))
        self.text_file.write("\nDatetime attribute count: %i" %(len(self.datetime)))

    def datetime_stats(self):
        try:
            self.date_time = pd.DataFrame(self.df[self.datetime].describe()).T
            self.date_time.to_csv('./datetime_stats.csv')
        except:
            pass

    def nominal_stats(self):
        self.df = self.df.replace('nan', np.nan)
        self.unique_val_count = []
        self.null_val_count = []
        self.val_counts = []
        for n in self.nominal:
            self.unique_val_count.append(self.df[n].nunique())
            self.null_val_count.append(self.df[n].isnull().sum())
            self.df['index'] = self.df.index
            self.sorted_counts = self.df.groupby(n)['index'].count()   
            self.df1 = pd.DataFrame(list(zip(list(self.sorted_counts.index), list(self.sorted_counts.values), [x/float(self.df.shape[0])*100 for x in list(self.sorted_counts.values)])), columns = ['attribute_values', 'value_count', 'value_count_%_from_total'])
            additional = ['Nulls', self.df.shape[0] - sum(self.df1['value_count']), 100 - sum(self.df1['value_count_%_from_total'])]
            self.df2 = pd.DataFrame([additional], columns=self.df1.columns)
            self.val_counts.append(self.df1.append(self.df2))

        self.null_pft = [x/float(self.df.shape[0])*100 for x in self.null_val_count]

        self.nominal_stats = pd.DataFrame(list(zip(self.unique_val_count, self.null_val_count, self.null_pft)), index= self.nominal, columns=['unique_val_count', 'null_val_count', 'null_%_from_total'])
        self.nominal_stats.to_csv('./nominal_stats.csv')

        for i in range(len(self.val_counts)):
            self.val_counts[i].to_csv('./'+self.nominal[i]+'_val_counts.csv', index=False)

    def numeric_stats(self):
        self.stats_numeric_values = self.df.describe()
        del self.stats_numeric_values['index']

        self.nulls = []
        self.median = []
        for n in self.numeric:
            self.nulls.append(self.df[n].isnull().sum())
            self.median.append(self.df[n].median())

        self.stats_numeric_values = self.stats_numeric_values.append(pd.DataFrame([self.nulls], index=['null_count'], columns= self.numeric), ignore_index=False)
        self.stats_numeric_values = self.stats_numeric_values.append(pd.DataFrame([self.median], index=['median'], columns= self.numeric), ignore_index=False)
        self.stats_numeric_values.T.to_csv('./numeric_stats.csv')

    def plot_numeric_histograms(self, type):
        if type == 'normal':
            for n in range(len(self.numeric)):
                plt.figure()
                self.df[self.numeric[n]].hist(bins=100)
                plt.title(self.numeric[n])
                plt.xlabel('Attribute values')
                plt.ylabel('Frequency')
                plt.savefig(self.numeric[n]+".png")
        else:
            for n in range(len(self.numeric)):
                try:
                    plt.figure()
                    self.df[self.numeric[n]].hist(bins=100)
                    plt.title(self.numeric[n])
                    plt.xscale('log')
                    plt.xlabel('Attribute values (log scale)')
                    plt.ylabel('Frequency')
                    plt.savefig(self.numeric[n]+"_logscale.png")
                except:
                    pass

    def close(self):
        os.chdir('..')
        self.text_file.close()

In [50]:
datatables = ['calendar', 'campaigns', 'customers', 'orderlines', 'orders', 'products', 'reviews', 'subscribers', 
              'zipcensus', 'zipcounty']
for datatable in datatables:
    p = prelimstats(datatable)
    p.dbconnect()
    p.metadata_overview()
    p.datetime_stats()
    p.nominal_stats()
    p.numeric_stats()
    p.plot_numeric_histograms('normal')
    p.plot_numeric_histograms('log')
    p.close()

C:\Users\Orysya\Desktop\DSE203\zipcountry\zipcounty


C:\Users\Orysya\Anaconda\envs\py36\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [52]:
!pwd
# %cd ..

C:\Users\Orysya\Desktop\DSE203\zipcountry
C:\Users\Orysya\Desktop\DSE203
